In [1]:
import torch

In [2]:
torch.__version__

'0.4.0'

In [3]:
import numpy as np
from torch import nn, optim
from torchvision import datasets, transforms, models
import torch.nn.functional as F
import torchvision

In [4]:
device = torch.device('cpu')

In [5]:
vgg19_bn = models.vgg19_bn(pretrained=True).to(device)

RuntimeError: cuda runtime error (2) : out of memory at /opt/conda/conda-bld/pytorch_1524584710464/work/aten/src/THC/THCTensorRandom.cu:25

In [5]:
vgg1 = nn.Sequential(vgg19_bn.features[:-1])

$448 \rightarrow 28$
$416 \rightarrow 26$
$224 \rightarrow 14$

Confidence should be considered as a binary classification problem. Now to get confidence of that prediction we can make a probabilistic model.
A probabilistic neural network learns the distribution of its weights and biases. To get a prediction we sample $N$ neural nets and find mean and std of their predictions. The mean is considered as final prediction and the std is considered as the certainity.

Currently using 448 resolution. So that, that doesn't affect accuracy. In total there are 4 conv layers on top of vgg. Not going to train vgg layers.

Find out how to resize to B x C x H x W (cv2), check the classifier output
+ https://github.com/ayooshkathuria/pytorch-yolo-v3/blob/master/preprocess.py
+ Use padding, don't change the aspect ratio, use open-cv to resize
+ use transpose

How to resize boxes accordingly.
+ That is also done. Refer to detect.py
+ Box in the original grid cell (before resizing) would still be in the same cell after resizing
    + So, the relative position of its centre will also be the same.
    + Also, the relative width and height of box would remain the same
    + So image height and width also need to be attached to the annotations.
+ What about augmentation ?

Do we use non-max suppression on yolo or not.
+ First the predictions are filtered based on confidence, then non max is applied. But it is optional

That repo contains several useful things like yolo cfg file.
+ But it does not containing the training script

In [ ]:
for n, c in vgg1.named_children():
    for n2, child in c.named_children():
        for n3, params in child.named_parameters():
            params.requires_grad = False

In [24]:
vgg1.add_module(str(52), nn.Conv2d(512,512, kernel_size=(3,3)))
vgg1.add_module(str(53), nn.BatchNorm2d(512))
vgg1.add_module(str(54), nn.LeakyReLU(inplace=True))
vgg1.add_module(str(55), nn.Conv2d(512,256, kernel_size=(3,3), stride=(2,2), padding=(1,1)))
vgg1.add_module(str(56), nn.BatchNorm2d(256))
vgg1.add_module(str(57), nn.LeakyReLU(inplace=True))
vgg1.add_module(str(58), nn.Conv2d(256,512, kernel_size=(1,1)))
vgg1.add_module(str(59), nn.BatchNorm2d(512))
vgg1.add_module(str(60), nn.LeakyReLU(inplace=True))
vgg1.add_module(str(61), nn.Conv2d(512,20, kernel_size=(3,3), padding=(1,1)))
vgg1.add_module(str(63), nn.Sigmoid())

In [25]:
x = torch.rand((1,3,448,448))

In [26]:
pred = vgg1.forward(x)

In [27]:
pred.size()

torch.Size([1, 20, 13, 13])

In [29]:
pred = pred.permute(0, 2, 3, 1).view(-1, 169, 20)

In [30]:
pred_np = pred.data.numpy()

In [31]:
pred_np.shape

(1, 169, 20)

Somehow get the annotations and the image dimensions

In [ ]:
img_w, img_h = 920, 640 #data['image_dims']

In [ ]:
img_sq_a = max(img_w, img_h)
grid_a = img_sq_a / 13
for box in boxes:
    x,y, w, h = box
    cell_x = np.floor(x / grid_a)
    offset_x = (x / grid_a) - cell_x
    cell_y = np.floor(y / grid_a)
    offset_y = (y / grid_a) - cell_y
    rel_w = w / img_sq_a
    rel_h = h / img_sq_a
    gt[cell_y, cell_x].append([offset_x, offset_y, rel_w, rel_h])

#numpy op
x, y, w, h = boxes[:, 0], boxes[:, 1], boxes[:, 2], boxes[:, 3]
cell_x = np.floor(x / grid_a)
offset_x = (x / grid_a) - cell_x
cell_y = np.floor(y / grid_a)
offset_y = (y / grid_a) - cell_y
rel_w = w / img_sq_a
rel_h = h / img_sq_a
idx = cell_x * 13 + cell_y
gt = np.zeros((169,20), dtype='float32')
offset_x = offset_x.reshape(-1,1)
offset_y = offset_y.reshape(-1,1)
rel_w = rel_w.reshape(-1,1)
rel_h = rel_h.reshape(-1,1)

for i in range(169):
    # one cell might have many ground truth boxes
    box_ind = np.where(idx.reshape(-1) == i)[0]
    boxes_in_cell_i = np.concatenate([offset_x[box_ind], offset_y[box_ind], rel_w[box_ind], rel_h[box_ind]], axis =-1)
    # only 5 ground truth boxes can be taken at max
    boxes_in_cell_i = boxes_in_cell_i[:5]
    boxes_in_cell_i = boxes_in_cell_i.reshape(-1)
    gt[i, :len(boxes_in_cell)] = boxes_in_cell_i

Prepare a square box. Find relative center coord and width and heigth of each bbox

In [9]:
x = np.array([ [0.1 , 0.2, 0.3],
             [0.4, 0.5, 0.6],
             [0.7, 0.8, 0.9]])

In [8]:
np.where(x.reshape(-1) < 0.5)[0]

array([0, 1, 2, 3])

In [12]:
x.reshape(-1)

array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

In [8]:
np.floor(12.5 / 5.01)

2.0

In [15]:
a = np.zeros((13,3))

In [16]:
b = np.ones((6,3))

In [17]:
a[[2,3,6,9,5,1]] = b; a

array([[0., 0., 0.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [0., 0., 0.],
       [1., 1., 1.],
       [1., 1., 1.],
       [0., 0., 0.],
       [0., 0., 0.],
       [1., 1., 1.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])